In [1]:
import os
import tensorflow as tf

os.sys.path.append('/home/subin/AR3D_SVR2/AR3D_tf/')

from config_tf import config
from dataloader_tf import load_data, DataLoader
from network import AR3D_tf as AR3D


# os.environ["CUDA_VISIBLE_DEVICES"]="0, 1, 2"
os.environ["CUDA_VISIBLE_DEVICES"]=''

2021-07-23 19:46:15.542843: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [3]:
import numpy as np

x_train, y_train = load_data(config.dataset, "train")
x_valid, y_valid = load_data(config.dataset, "val")

train_loader = DataLoader(x_train, y_train, 1, 'train', config.batch_size)
test_loader =  DataLoader(x_train, y_train, 1, 'test', config.batch_size)

model = AR3D.AR3D(num_classes = config.num_classes, AR3D_V = config.ar3d_version, SFE_type = config.sfe_type, 
                    attention_method = config.attention, reduction_ratio = config.reduction_ratio, hidden_unit = config.hidden_units)
opt = tf.keras.optimizers.Adam(learning_rate=config.lr)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape=(config.batch_size, config.frames_per_clips, 112, 112, 3))

model.summary()


# if config.model_name == 'AR3D':
    
#     ## multi gpu test1 
#     stratege = tf.distribute.MirroredStrategy()

#     with stratege.scope():
#         x_train, y_train
#         model = AR3D.AR3D(num_classes = config.num_classes, AR3D_V = config.ar3d_version, SFE_type = config.sfe_type, 
#                             attention_method = config.attention, reduction_ratio = config.reduction_ratio, hidden_unit = config.hidden_units)
#         model.build(input_shape=(config.batch_size, config.frames_per_clips, 112, 112, 3))
#         opt = tf.keras.optimizers.Adam(learning_rate=config.lr)
#         model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    


Model: "a_r3d_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sfe_block_1 (sfe_block)      multiple                  13499136  
_________________________________________________________________
r3d_module_1 (r3d_module)    multiple                  332160    
_________________________________________________________________
a3d_module_1 (a3d_module)    multiple                  263040    
_________________________________________________________________
dense_3 (Dense)              multiple                  205524992 
_________________________________________________________________
dense_4 (Dense)              multiple                  16781312  
_________________________________________________________________
dense_5 (Dense)              multiple                  413797    
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0   

In [7]:
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.Accuracy()
test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.Accuracy()

epochs = 50
for epoch_index in range(epochs):
    for x_train_batch, y_train_batch in train_loader:
        with tf.GradientTape() as tape:
            predictions = model(x_train_batch, training=True)
            loss_value = loss(y_train_batch, predictions)
        gradients = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss.update_state(loss_value)
        train_accuracy.update_state(tf.argmax(y_train_batch, axis=1), tf.argmax(predictions, axis=1))

    for x_test_batch, y_test_batch in test_loader:
        predictions = model(x_test_batch)
        loss_value = loss(y_test_batch, predictions)
        test_loss.update_state(loss_value)
        test_accuracy.update_state(tf.argmax(y_test_batch, axis=1), tf.argmax(predictions, axis=1))

    print('epoch: {}/{}, train loss: {:.4f}, train accuracy: {:.4f}, test loss: {:.4f}, test accuracy: {:.4f}'.format(
        epoch_index + 1, epochs, train_loss.result().numpy(), train_accuracy.result().numpy(), test_loss.result().numpy(), test_accuracy.result().numpy()))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

KeyboardInterrupt: 